## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [2]:
standard_settings = dict(Risk_of_exceedance='50%',
                         Negative_emissions='Medium',
                         Non_CO2_mitigation_potential='Medium',)

In [3]:
params = dict(start_year_analysis=2021)

In [4]:
rules = ['GF', 'PC', 'PCC', 'AP', 'GDR']
rulecolors = ['steelblue', 'goldenrod', 'forestgreen', 'sienna', 'mediumvioletred']
costoptimal_colors = ['orangered', 'red', 'darkred']

## Paths

In [5]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data"
path_figs = path_main / "Figures" / "ECEMF_paper"

## Read data files

In [6]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

In [7]:
df = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso2 = np.array(df["Country ISO Code"])
group_ldc = countries_iso2[np.array(df["LDC"]) == 1]
group_hdc = countries_iso2[np.array(df["HICO"]) == 1]

Find out high/medium/low income

In [8]:
group_mdc = []
for reg in all_countries_iso:
    if reg not in group_hdc and reg not in group_ldc:
        group_mdc.append(reg)

In [9]:
def get_max_index(arr):
    return np.nanargmax(arr)

def get_values(group, temperature):
    values = []
    for reg in group:
        ar = np.array([
            float(xr_total.sel(Temperature=temperature, **standard_settings, Region=reg, Convergence_year=2100, Scenario='SSP2', Time=2030)[rule])
            for rule in rules
        ])
        values.append(get_max_index(ar))
    return values

high_vals = get_values(group_hdc, '1.5 deg')
med_vals = get_values(group_mdc, '1.5 deg')
low_vals = get_values(group_ldc, '1.5 deg')

## Plot

In [25]:
fig = make_subplots(rows=4, cols=6,
                    specs=[[{'colspan': 3, 'rowspan': 2}, {}, {}, {'colspan': 3, 'rowspan': 2,'type': 'choropleth'}, {}, {}],
                           [{}, {}, {}, {}, {}, {}],
                           [{'colspan': 2}, {}, {'colspan': 2}, {}, {'colspan': 2}, {}],
                           [{'colspan': 2,'type': 'choropleth'}, {}, {'colspan': 2,'type': 'choropleth'}, {}, {'colspan': 2,'type': 'choropleth'}, {}]],
                    horizontal_spacing = 0.02, vertical_spacing=0.06)

fig.update_layout(height=1300,
                  width=1600,
                  template='simple_white')

# PANEL (A), (C), (D), (E)
lw = 4
for ax_i in range(4):
       regs = [countries_iso, group_hdc, group_mdc, group_ldc][ax_i]
       row = [1, 3, 3, 3][ax_i]
       col = [1, 1, 3, 5][ax_i]
       if ax_i == 0: sl = True
       else: sl=False

       # Historical emissions
       fig.add_trace(go.Scatter(x=np.arange(1990, params['start_year_analysis']+1),
                            y=xr_total.sel(Time=np.arange(1990, params['start_year_analysis']+1), Region=regs).CO2_hist.sum(dim='Region')/1e3,
                            name='Historical emissions',
                            line={'color': 'silver', 'width': lw},
                            mode='lines',
                            showlegend=sl), row, col)
        # Baseline emissions
       fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                y=xr_total.sel(Time=np.arange(params['start_year_analysis'], 2101), Scenario=['SSP1', 'SSP2'], Region=regs, Temperature='1.5 deg', **standard_settings).CO2_base.sum(dim='Region').min(dim='Scenario')/1e3,
                                name='SSP2-baseline',
                                fill=None,
                                line={'color': 'grey', 'width': lw/4},
                                mode='lines',
                                showlegend=False), row, col)
       fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                y=xr_total.sel(Time=np.arange(params['start_year_analysis'], 2101), Scenario=['SSP1', 'SSP2'], Region=regs, Temperature='1.5 deg', **standard_settings).CO2_base.sum(dim='Region').max(dim='Scenario')/1e3,
                                name='SSP2-baseline',
                                line={'color': 'grey', 'width': lw/4},
                                mode='lines',
                                fill='tonexty',
                                showlegend=sl), row, col)

        # Allocations
       if ax_i == 0:
              for temp_i, temp in enumerate(xr_total.Temperature):
                     fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                              y=xr_total.min(dim=['Scenario', 'Convergence_year']).sel(Time=np.arange(params['start_year_analysis'], 2101), Temperature=temp, **standard_settings).CO2_globe/1e3,
                                              name="Cost-optimal: "+['1.5 degrees', '1.7 degrees', '2 degrees'][temp_i],
                                              line={'color': costoptimal_colors[temp_i], 'width': lw/2},
                                              mode='lines',
                                              fill=None,
                                              showlegend=False), row, col)
                     fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                              y=xr_total.max(dim=['Scenario', 'Convergence_year']).sel(Time=np.arange(params['start_year_analysis'], 2101), Temperature=temp, **standard_settings).CO2_globe/1e3,
                                              name="Cost-optimal: "+['1.5 degrees', '1.7 degrees', '2 degrees'][temp_i],
                                              line={'color': costoptimal_colors[temp_i], 'width': lw/2},
                                              mode='lines',
                                              fill='tonexty',
                                              showlegend=sl), row, col)
       else:
              for var_i, var in enumerate(rules):
                     fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                              y=xr_total.min(dim=['Scenario', 'Convergence_year']).sel(Time=np.arange(params['start_year_analysis'], 2101), Region=regs, Temperature=temp, **standard_settings)[var].sum(dim=['Region'])/1e3,
                                              name=['Grandfathering', 'Per capita', 'Per capita convergence', 'Ability to pay', 'Greenhouse Development Rights', 'Equal Cumulative per capita'][var_i],
                                              line={'color': rulecolors[var_i], 'width': lw/2},
                                              mode='lines',
                                              fill=None,
                                              showlegend=False), row, col)
                     fig.add_trace(go.Scatter(x=np.arange(params['start_year_analysis'], 2101),
                                              y=xr_total.max(dim=['Scenario', 'Convergence_year']).sel(Time=np.arange(params['start_year_analysis'], 2101), Region=regs, Temperature=temp, **standard_settings)[var].sum(dim=['Region'])/1e3,
                                              name=['Grandfathering', 'Per capita', 'Per capita convergence', 'Ability to pay', 'Greenhouse Development Rights', 'Equal Cumulative per capita'][var_i],
                                              line={'color': rulecolors[var_i], 'width': lw/2},
                                              mode='lines',
                                              fill='tonexty',
                                              showlegend=[False, True, False, False][ax_i]), row, col)

              fig.add_trace(
                     go.Choropleth(
                     locations=regs,
                     z = [high_vals, med_vals, low_vals][ax_i-1],
                     locationmode = 'ISO-3',
                     colorscale = rulecolors,
                     zmax = 4,
                     zmin = 0,
                     hovertemplate  = '%{text}',
                     name="",
                     marker_line_color='black', 
                     marker_line_width=0.35,
                     showscale=False
                     ), row+1, col)
       
       fig.add_trace(go.Scatter(x=[2030],
                            y=[xr_total.CO2_ndc.sel(Region=regs).sum(dim='Region').sel(Time=2030).max()/1e3],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines+markers',
                            marker={'size': 12, 'symbol': 'triangle-down'},
                            showlegend=False), row, col)
       
       fig.add_trace(go.Scatter(x=[2030],
                            y=[xr_total.CO2_ndc.sel(Region=regs).sum(dim='Region').sel(Time=2030).min()/1e3],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines+markers',
                            marker={'size': 12, 'symbol': 'triangle-up'},
                            showlegend=False), row, col)
       
       fig.add_trace(go.Scatter(x=[2030, 2030],
                            y=[xr_total.CO2_ndc.sel(Region=regs).sum(dim='Region').sel(Time=2030).min()/1e3,
                               xr_total.CO2_ndc.sel(Region=regs).sum(dim='Region').sel(Time=2030).max()/1e3],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines',
                            marker={'size': 12, 'symbol': 'triangle-up'},
                            showlegend=sl), row, col)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.76,
    xanchor="left",
    x=0.01,
    font=dict(
        size=11,
        color="black"
    ),
))
fig.update_yaxes(title_text="Global CO2 emissions [Gt CO2/yr]", row=1, col=1)

# PANEL (B)
standard_settings_ndc = dict(Hot_air='include',
                             Conditionality='unconditional')
ndc = np.array(xr_total.CO2_ndc.sel(Time=2030, Region=all_countries_iso, **standard_settings_ndc).mean(dim='Ambition'))
base = np.array(xr_total.CO2_base.sel(Time=2030, Region=all_countries_iso).mean(dim='Scenario'))
t = (ndc-base)/base
fig.add_trace(
       go.Choropleth(
       locations=all_countries_iso,
       z = t,
       locationmode = 'ISO-3',
       colorscale = 'RdBu_r',
       zmax = 1,
       zmin = -1,
       text = [str(r)+": "+str(np.round(float(t[r_i]*100), 2))+"%" for r_i, r in enumerate(all_countries_iso)],
       hovertemplate  = '%{text}',
       name="",
       marker_line_color='black', 
       marker_line_width=0.5,
       colorbar=dict(len=0.32, x=0.76, y=0.49, orientation='h',
                  tickvals = np.arange(-1, 1.05, 0.5),
                  thickness=15,
                  tickfont={'size': 13},
                  ticktext=['-100%<br>(emission reduction<br>w.r.t. baseline)', '-50%', '0%', '50%', '100%<br>(emission increase<br>w.r.t. baseline)'],
                  ticklabelposition='outside',
    ),
       showscale=True
), 1, 4)

fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")
fig.update_geos(visible=True,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                coastlinecolor='silver',
                projection_type='natural earth',
                showcoastlines=True)
fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=True, row=1, col=4)

fig.update_yaxes(title_text="GHG emissions [Gt CO2e/yr]", row=3, col=1)
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.97, x=0.01, text="<b>(a) Global emission trajectories</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.97, x=0.90, text="<b>(b) NDC emissions in 2030 w.r.t. baseline</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.01, text="<b>(c) High-income countries</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.46, text="<b>(d) Medium-income countries</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.88, text="<b>(e) Low-income countries</b>")
fig.show()

In [26]:
fig.write_image(path_figs / "Figure_1.png", scale=3)